In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
import $ivy.`com.github.dzufferey::scadla:0.1.1`

In [ ]:
import scadla._
import InlineOps._
import EverythingIsIn.{millimeters, degrees}
import scadla.utils.thread.ISO
import scadla.utils.CenteredCube
import squants.space.Length
import scala.language.postfixOps
import squants.space.LengthConversions._
import scadla.backends.OpenSCAD // for rendering (getting a mesh)
import scadla.backends.almond.Viewer // to show the mesh in jupyter/almond

# New Z-axis for my cnc

## Dimensions for mechanical components

In [ ]:
// linear rails (MGN 15C)
object LinearRail {
    val totalHeight = 16 mm // surface to surface distance includes rail + guide
    val width = 15 mm
    val height = 10 mm
    val holesDistance = 40 mm
    val holesRadius = ISO.M3
    def screwPositions(length: Length) = { // x coord of the mounting screw
        val n = ((length - holesRadius * 2) / holesDistance).floor.toInt
        val offset = (length - holesDistance * n) / 2
        (0 until n).map( i => offset + holesDistance * i)
    }
    // extend in x direction [0, length], y centered at 0, back against xy plane
    def rail(length: Length) = {
        val r = Cube(length, width, height).moveY(- width / 2)
        val screw = Cylinder(holesRadius, height) + Cylinder(holesRadius * 2, holesRadius).moveZ(height - holesRadius + 0.1)
        val hs = screwPositions(length).map( x => screw.moveX(x) )
        r -- hs
    }
    def holePattern(length: Length, h: Length = 10) = {
        val screw = Cylinder(holesRadius, h+1).moveZ(-h)
        screwPositions(length).map( x => screw.moveX(x) )
    }
}
object LinearGuide {
    val length = 42.1 mm
    val width = 32 mm
    val height = 12 mm
    val holesLength = 20 mm
    val holesWidth = 25 mm
    val holesRadius = ISO.M3
    // centered at (0,0), back at 0 and guide facing up
    val holePattern = Seq(( holesLength / 2,  holesWidth / 2),
                          (-holesLength / 2,  holesWidth / 2),
                          ( holesLength / 2, -holesWidth / 2),
                          (-holesLength / 2, -holesWidth / 2))
    val model = {
        val c0 = CenteredCube.xy(length, width, height)
        val c1 = c0 - CenteredCube.xy(length, LinearRail.width, LinearRail.height).moveZ(4)
        c1 -- holePattern.map{ case (x,y) => Cylinder(holesRadius, height).move(x, y, 0) } // make througholes to see align
    }
}
// ballscrew
object BallScrew {
    // https://www.dold-mechatronik.de/Spindelmuttergehaeuse-DSG1605-fuer-Kugelumlaufspindel-SFU1605-SFU1610
    val nutHeight = 40 mm
    val nutWidth = 52 mm
    val nutLength = 40 mm
    val nutHoles = ISO.M5
    val nutHolesDistanceWidth = 40 mm
    val nutHolesDistanceLength = 24 mm
    val nutHolesPattern = Seq(( nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              ( nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2))
    // https://www.dold-mechatronik.de/Festlagereinheit-FLB20-3200-Easy-Mechatronics-System-1620B
    val endHeight = 51 mm
    val endWidth = 70 mm
    val endDepth = 20 mm
    val endHolesDistance = 55 mm
    val endHoles = ISO.M6
    val endHolesPattern = Seq(-endHolesDistance / 2, endHolesDistance / 2)
    val bottomToRotationAxis = 31 mm
    val bearingUnit = {
        val c0 = CenteredCube.xy(endDepth, 40, endHeight)
        val c1 = c0 + CenteredCube.xy(endDepth, endWidth, 10)
        val c2 = c1 -- endHolesPattern.map( y => Cylinder(endHoles, 11).moveY(y) )
        c2
    }
    // length is the distance between mounting screw
    def model(l: Length) = {
        bearingUnit + bearingUnit.moveX(l) + Cylinder(10, l).rotateY(90).moveZ(bottomToRotationAxis)
    }
}
// spindle clamp: https://www.dold-mechatronik.de/Eurohalsaufnahme-65x75x20
object SpindleClamp {
    val width = 65 mm
    val height = 20 mm
    val depth = 75 mm
    val backToCenter = 42 mm
    val centerDiameter = 43 mm
    val centerGap = 2 mm
    val mountingHoles = ISO.M8
    val holesPosition = Seq(7.5 mm, 32.5 mm, 57.5 mm)
    val holesHeight = 10 mm
    val model = {
        val x0 = Cube(width, depth, height)
        val x1 = x0 - Cylinder(centerDiameter/2, height).move(width/2, backToCenter, 0)
        val x2 = x1 - CenteredCube.x(centerGap, depth, height).move(width/2, backToCenter + 1, 0)
        val screw = Cylinder(mountingHoles, 17).rotateX(-90)
        val x3 = x2 -- holesPosition.map( x => screw.move(x, 0, height / 2) )
        x3
    }
}

def holesPosition0(n: Int, start: Length, end: Length) = {
    val delta = (end - start) / (n-1)
    (0 until n).map( i => start + delta * i )
}

def holesPosition1(n: Int, length: Length, distanceToBorder: Length) = {
    holesPosition0(n, distanceToBorder, length - distanceToBorder)
}

## XY table (already made)

In [ ]:
object Table {
    val mountingHolesWidth = 160 mm
    val mountingHolesDepth = 80 mm
    val mountingHoles = ISO.M8
    val xTravel = 300 mm
    val yTravel = 200 mm
    val distBetweenPlates = BallScrew.nutHeight / 2 +  BallScrew.bottomToRotationAxis
    val plateThickness = 20 mm
    val totalHeigth = plateThickness * 3 + distBetweenPlates * 2
    def placeholder(x: Double = 0.5, y: Double = 0.5) = {
        val lower = CenteredCube.xy(550, 200, plateThickness)
        val mz = plateThickness + distBetweenPlates
        val xPos = xTravel * (0.5 - x)
        val middle = CenteredCube.x(200, 350, plateThickness).move(xPos, -130, mz)
        val uz = mz * 2
        val yPos = yTravel * (1 - y)
        val upper = CenteredCube.xy(300, 200, plateThickness).move(xPos, yPos - 55, uz)
        lower + middle + upper
    }
    val xOffset = 0 mm
    val yOffset = 45 mm
}

In [ ]:
val mesh = OpenSCAD(Table.placeholder())
Viewer(mesh)

## Draft

In [ ]:
val carriageHeight = 150 mm
val pillarInnerspace = BallScrew.nutWidth.max(BallScrew.endWidth) + 10 //TODO smaller endwidth
val expectedTravel = 150 mm

//TODO where to put the z-axis stop

object Pillar {
    val width = 15 mm
    val depth = 60 mm
    val height = Table.totalHeigth +
                 15 /* slots for t-nuts */ +
                 30 + /* spindle collet to collar */
                 carriageHeight + /* carriage heigth */
                 expectedTravel /* expected z travel */
    val railLength = height
    
    val kneeDepth = 50 mm
    val kneeHeight = 20 mm
    
    val baseHoles = ISO.M8
    val baseHolesNumber = 3
    val baseHolesPositions = {
        val l = depth + kneeDepth
        val toBorder = baseHoles * 2.5
        val dist = (l - toBorder * 2) / (baseHolesNumber - 1)
        Seq(toBorder, toBorder + dist + 5, toBorder + dist * 2)
    }
    
    val frontDepth = 6 mm
    val frontHole = ISO.M3
    val frontHeight = height - kneeHeight
    val frontHolesPositions = holesPosition1(6, frontHeight, frontHole * 3 + 10).map( _ + kneeHeight )
    
    //TODO should fillet the knee
    //TODO rail length should be somewhat standard
    
    def model = {
        val p = Cube(width, depth, height)
        val k = Cube(width, kneeDepth, kneeHeight)
        val s0 = p + k.moveY(-kneeDepth)
        // holes to attach
        // (1) the base
        val s1 = s0 -- baseHolesPositions.map( y => Cylinder(baseHoles, 15).move(width/2, y - kneeDepth, 0) )
        // (2) the front
        val s2 = s1 -- frontHolesPositions.map( z => Cylinder(frontHole, width).rotateY(90).move(0, frontDepth / 2 , z) )
        // (3) the rails
        val s3 = s2 -- LinearRail.holePattern(railLength,width).map( _.rotateY(-90).moveY(depth/2) )
        s3
    }
    
    def info = {
        s"""|Pillar (2x):
            |  width = $width
            |  depth = $depth
            |  height = $height
            |  rail length =  $railLength""".stripMargin
    }
}

In [ ]:
val mesh = OpenSCAD(Pillar.model)
Viewer(mesh)

In [ ]:
object FrontPlate {
    val width = pillarInnerspace
    val depth = Pillar.frontDepth
    val height = Pillar.height
    val ballScrewLength = expectedTravel + 200 // when I make a lower table, I'll have more travel
    
    // TODO spacer for the front plate to hold the ballscrew at the proper distance
    // TODO to mount the ballscrew attach from the back, tighter holes
    // TODO better travel / height ratio ?
    
    def model = {
        val f0 = Cube(width, depth, height)
        // holes for pillar
        val pd2 = depth / 2
        val s = Cylinder(Pillar.frontHole, 10).rotateY(90)
        val f1 = f0 -- Pillar.frontHolesPositions.map( z => s.move(0, pd2, z) + s.move(width-10, pd2, z))
        // holes for ballscrew
        val s2 = Cylinder(BallScrew.endHoles, 10).rotateX(-90)
        val h2 = height - 10
        val h3 = h2 - ballScrewLength + 10
        val f2 = f1 -- BallScrew.endHolesPattern.map( x => s2.move(width/2+x, 0, h2) + s2.move(width/2+x, 0, h3) )
        f2
    }
    
    def info = {
        s"""|FrontPlate (1x):
            |  width = $width
            |  depth = $depth
            |  height = $height
            |  ballScrewLength = $ballScrewLength""".stripMargin
    }
}

In [ ]:
val mesh = OpenSCAD(FrontPlate.model)
Viewer(mesh)

In [ ]:
object Base {
    val width = 200 mm
    val height = 20 mm
    val depth = (200 mm) /* table bottom plate */ +
                (115 mm) /* table middle plate overhang*/ +
                (20 mm) /* clearance from the top plate */ +
                Pillar.depth
    val tableMountingHoles = Seq((width/2 - Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 - Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2))
    def model = {
        val c0 = Cube(width, depth, height)
        // mounting holes for the table (should be threaded!)
        val tableScrew = Cylinder(Table.mountingHoles, height+1)
        val c1 = c0 -- tableMountingHoles.map{ case (x,y) => tableScrew.move(x,y,0) }
        // countersunk holes to mount the pillar
        val pillarScrew = Cylinder(Pillar.baseHoles, height+1) + Cylinder(Pillar.baseHoles * 2.2, Pillar.baseHoles + 1)
        val widthDifference = (pillarInnerspace + Pillar.width) / 2
        val c2 = c1 -- Pillar.baseHolesPositions.map( y => pillarScrew.move(width/2 + widthDifference, depth - y, 0))
        val c3 = c2 -- Pillar.baseHolesPositions.map( y => pillarScrew.move(width/2 - widthDifference, depth - y, 0))
        c3
    }
    
    def info = {
        s"""|Base (1x):
            |  width = $width
            |  depth = $depth
            |  height = $height""".stripMargin
    }
}

In [ ]:
val mesh = OpenSCAD(Base.model)
Viewer(mesh)

In [ ]:
object Carriage {
    val sideThickness = 10 mm
    val frontThickness = 10 mm
    val backThickness = 6 mm
    
    val frontBackClearance = 5 mm
    
    val height = carriageHeight
    val frontHeight = height
    val backHeight = BallScrew.nutLength
    val backHeightOffset = 50 mm
    val innerWidth = pillarInnerspace + Pillar.width * 2 + LinearRail.totalHeight * 2
    val outerWidth = innerWidth + sideThickness * 2
    val sideDepth = Pillar.depth +
                    frontBackClearance * 2
    val depth = sideDepth +
                frontThickness +
                backThickness
    val upperGuideHeight = LinearGuide.length / 2 + 5
    val lowerGuideHeight = height - upperGuideHeight
    val guideDepth = sideDepth / 2
    
    val frontBackHole = ISO.M4
    val frontHolesPositions = holesPosition1(4, frontHeight, frontBackHole * 3)
    val backHolesPositions = holesPosition1(3, backHeight, frontBackHole * 3)
    val frontBackScrew = Cylinder(frontBackHole, 40).moveZ(-20).rotateX(90)
    
    def side = {
        val s0 = Cube(sideThickness, sideDepth, height)
        val guideScrew = Cylinder(LinearGuide.holesRadius, height+2).rotateY(90)
        // holes to mount the guides
        val s1 = s0 -- LinearGuide.holePattern.map{ case (x,y) => guideScrew.move(-1, guideDepth + y, upperGuideHeight + x) }
        val s2 = s1 -- LinearGuide.holePattern.map{ case (x,y) => guideScrew.move(-1, guideDepth + y, lowerGuideHeight + x) }
        // holes to mount the front
        val s3 = s2 -- frontHolesPositions.map( z => frontBackScrew.move(sideThickness/2, 0, z))
        // holes to mount the back
        val s4 = s3 -- backHolesPositions.map( z => frontBackScrew.move(sideThickness/2, sideDepth, height - backHeight - backHeightOffset + z))
        s4
    }
        
    def back = {
        val h = backHeight
        val b0 = Cube(outerWidth, backThickness, h)
        // holes to attach to the ballscrew nut
        val nutScrew = Cylinder(BallScrew.nutHoles, backThickness).rotateX(-90)
        val b1 = b0 -- BallScrew.nutHolesPattern.map{ case (z,x) => nutScrew.move(outerWidth / 2 + x, 0, h / 2 + z) }
        // holes to attach to side
        val b2 = b1 -- backHolesPositions.map( z => frontBackScrew.move(sideThickness/2, 0, z) +
                                                    frontBackScrew.move(outerWidth - sideThickness/2, 0, z))
        b2
    }
    
    val cantileverWidth = SpindleClamp.width
    val cantileverDepth = Base.depth -
                          100 /* to middle of table */ -
                          Pillar.depth / 2 /* to linear rail */ -
                          depth / 2 /* front of carriage */ -
                          sideThickness /* frontplate */ -
                          SpindleClamp.backToCenter -
                          Table.yOffset
    val cantileverHeight = cantileverWidth // + backThickness
    val cantileverHoles = ISO.M5 // or M6 ?
    val cantileverHolesPositions = {
        val x = (cantileverWidth - sideThickness) / 2
        val z1 = sideThickness / 2 + 2
        val z3 = cantileverHeight - sideThickness / 2 - 2
        val z2 = (z1 + z3) / 2
        Seq((x,z1),(x,z2),(x,z3),(-x,z1),(-x,z2),(-x,z3))
    }
    
    def cantilever = {
        // 3 plates: 2 sides and top
        val side = Cube(sideThickness, cantileverDepth, cantileverHeight)
        val top = {
            val x = cantileverWidth
            val y = cantileverDepth + frontThickness * 2 //top goes over the plates where it is attached
            val z = backThickness - 2
            val c0 = Cube(x, y, z)
            val c1 = Cube(x - sideThickness*2, y - frontThickness*2, 2)
            c0 + c1.move(sideThickness, frontThickness, -2)
        }
        val shape = side.moveX(-cantileverWidth/2) +
                    side.moveX(cantileverWidth/2 - sideThickness) +
                    top.move(-cantileverWidth/2, -frontThickness, cantileverHeight)
        // holes on the back + front of the sides
        val s0 = Cylinder(cantileverHoles, 30).moveZ(-15).rotateX(90)
        val c0 = shape -- cantileverHolesPositions.map{ case(x,z) => s0.move(x,0,z) + s0.move(x,cantileverDepth,z) }
        // holes to attach the top
        val s1 = Cylinder(ISO.M3, 10).moveZ(cantileverHeight + frontThickness - 10)
        val x = (cantileverWidth  - sideThickness) / 2
        val y0 = 20 mm // TODO as param
        val y1 = cantileverDepth - 20
        val pos = Seq((x,y0),(x,y1),(-x,y0),(-x,y1))
        val c1 = c0 -- pos.map{ case (x,y) => s1.move(x,y,0) }
        c1
    }
    
    def front = {
        val f0 = Cube(outerWidth, frontThickness, frontHeight)
        // remove material to go over the pillar's knee
        val knee = pillarInnerspace + Pillar.width * 2 + 10
        val f1 = f0 - Cube(knee, frontThickness, Pillar.kneeHeight + 5).moveX((outerWidth - knee) / 2)
        // holes to attach to side
        val f2 = f1 -- frontHolesPositions.map( z => frontBackScrew.move(sideThickness/2, 0, z) +
                                                     frontBackScrew.move(outerWidth - sideThickness/2, 0, z))
        // holes to attach the cantilever
        val s = Cylinder(cantileverHoles, frontThickness).rotateX(-90)
        val f3 = f2 -- cantileverHolesPositions.map{ case (x,z) => s.move(outerWidth/2+x, 0, frontHeight - cantileverHeight + z) }
        f3
    }
    
    def spindlePlate = {
        val c0 = Cube(cantileverWidth, sideThickness, cantileverHeight + SpindleClamp.height)
        // attach to cantilever
        val s1 = Cylinder(cantileverHoles, frontThickness).rotateX(-90)
        val c1 = c0 -- cantileverHolesPositions.map{ case (x,z) => s1.move(cantileverWidth/2+x, 0, SpindleClamp.height + z) }
        // holes for SpindleClamp
        val s2 = Cylinder(SpindleClamp.mountingHoles, frontThickness).rotateX(-90)
        val c2 = c1 -- SpindleClamp.holesPosition.map( x => s2.move(x,0,SpindleClamp.height/2))
        // TODO extra holes to mount other stuff (on the side ?)
        c2
    }
    
    def model = {
        val sideAndGuides = {
            val g = LinearGuide.model.rotateY(-90).rotateZ(180).moveX(sideThickness)
            val s = side + g.move(0, guideDepth, lowerGuideHeight) + g.move(0, guideDepth, upperGuideHeight)
            s.move(-outerWidth/2, 0, 0)
        }
        val spindle = spindlePlate + SpindleClamp.model.rotateZ(180).moveX(SpindleClamp.width)
        sideAndGuides +
        sideAndGuides.mirror(1,0,0) +
        front.move(-outerWidth/2, -frontThickness, 0) +
        back.move(-outerWidth/2, sideDepth, height - backHeight - backHeightOffset) +
        cantilever.move(0, -cantileverDepth -frontThickness, height - cantileverHeight) +
        spindle.move(-cantileverWidth/2, -cantileverDepth - frontThickness - sideThickness, height - (cantileverHeight + SpindleClamp.height))
    }
    
    def info = {
        s"""|Carriage (assembly):
            |  side (2x)
            |    width = $sideThickness
            |    depth = $sideDepth
            |    height = $height
            |  front (1x)
            |    width = $outerWidth
            |    depth = $frontThickness
            |    height = $frontHeight
            |  back (1x)
            |    width = $outerWidth
            |    depth = $backThickness
            |    height = $backHeight
            |  cantilever (assembly)
            |    side (2x)
            |      width = $sideThickness
            |      depth = $cantileverDepth
            |      height = $cantileverHeight
            |    top (1x)
            |      width = $cantileverWidth
            |      depth = ${cantileverDepth + frontThickness * 2}
            |      height = $backThickness
            |  spindle plate (1x)
            |    width = $cantileverWidth
            |    depth = $sideThickness
            |    height = ${cantileverHeight + SpindleClamp.height}""".stripMargin
    }
}
//TODO splindle plate should be thicker and have countersunk screw ? or go for some kind of angle brackets / side bar ?
//TODO countersunk on the front plate
//TODO save some material on the front plate

In [ ]:
val mesh = OpenSCAD(Carriage.model)
Viewer(mesh)

### Together

In [ ]:
val base = Base.model
val column = {
    val pr = Pillar.model + LinearRail.rail(Pillar.railLength).rotateY(-90).moveY(Pillar.depth/2)
    val p = pr.move(-pillarInnerspace / 2 - Pillar.width, 0, 0)
    p + p.mirror(1,0,0) +
    FrontPlate.model.move(-FrontPlate.width/2, 0, 0) +
    BallScrew.model(FrontPlate.ballScrewLength).rotateY(-90).rotateZ(-90).moveZ(FrontPlate.height - 10 - FrontPlate.ballScrewLength)
}
val carriage = Carriage.model
val table = Table.placeholder()
val cnc = {
    base.move(-Base.width / 2, 0 , -Base.height) +
    column.move(0, Base.depth - Pillar.depth, 0) +
    carriage.move(0, Base.depth - Carriage.sideDepth + Carriage.frontBackClearance, 200) +
    table.move(0,100,0)
}

In [ ]:
val mesh = OpenSCAD(cnc)
Viewer(mesh)

### List of parts

In [ ]:
println(Base.info)
println(Pillar.info)
println(FrontPlate.info)
println(Carriage.info)

## Gearbox to hold the motor and handwheel

In [ ]:
// TODO import bevel gear